In [9]:
from desc.transform import Transform
from desc.equilibrium import Equilibrium
from desc.basis import ChebyshevZernikeBasis, chebyshev_z
from desc.compute import compute
from desc.grid import LinearGrid, ConcentricGrid, QuadratureGrid
from desc.compute.utils import get_transforms
from desc.objectives import (
    FixEndCapLambda,
    FixEndCapR,
    FixEndCapZ,
    FixBoundaryR,
    FixBoundaryZ,
    FixPsi,
    FixPressure,
    FixIota,
    ForceBalance,
    ObjectiveFunction,
)
import numpy as np
from desc.optimize import Optimizer
from desc.profiles import PowerSeriesProfile

# Import tests

In [17]:
p = PowerSeriesProfile(params=[0, 0], modes=[0, 2])
iota = PowerSeriesProfile(params=[0, 0])
eq = Equilibrium(L=3, M=3, N=3, mirror=True, pressure=p, iota=iota, sym=False, method="jitable")

In [18]:
constraints = (
        FixEndCapLambda(0, eq=eq),
        FixEndCapR(0, eq.eq),
        FixEndCapZ(0, eq=eq),
        FixEndCapLambda(2*np.pi, eq=eq),
        FixEndCapR(2*np.pi, eq=eq),
        FixEndCapZ(2*np.pi, eq=eq),
        FixBoundaryR(eq=eq),
        FixBoundaryZ(eq=eq),
        FixPressure(eq=eq),
        FixIota(eq=eq),
        FixPsi(eq=eq),
    )
optimizer = Optimizer("lsq-exact")
objectives = ForceBalance(eq=eq)
obj = ObjectiveFunction(objectives=objectives)

In [19]:
eq.solve(objective=obj, constraints=constraints,optimizer=optimizer)

Building objective: force
Precomputing transforms
Compiling objective function and derivatives: ['force']
Number of parameters: 32
Number of objectives: 728
Starting optimization
Using method: lsq-exact
`gtol` condition satisfied.
         Current function value: 9.554e-29
         Total delta_x: 0.000e+00
         Iterations: 0
         Function evaluations: 1
         Jacobian evaluations: 1
Start of solver
Total (sum of squares):  1.550e-33, 
Maximum absolute Force error:  5.355e-11 (N)
Minimum absolute Force error:  7.190e-13 (N)
Average absolute Force error:  9.395e-12 (N)
Maximum absolute Force error:  3.365e-18 (normalized)
Minimum absolute Force error:  4.518e-20 (normalized)
Average absolute Force error:  5.903e-19 (normalized)
End of solver
Total (sum of squares):  9.554e-29, 
Maximum absolute Force error:  2.485e-08 (N)
Minimum absolute Force error:  2.942e-12 (N)
Average absolute Force error:  2.755e-09 (N)
Maximum absolute Force error:  1.561e-15 (normalized)
Minimum absol

(Equilibrium at 0x2a2035590 (L=3, M=3, N=3, NFP=1, sym=False, spectral_indexing=ansi),
     message: `gtol` condition satisfied.
     success: True
         fun: [ 1.725e-15  1.888e-16 ...  5.151e-16  1.332e-16]
           x: [ 2.276e+00  5.379e+00 ...  4.705e-01 -8.155e-01]
         nit: 0
        cost: 9.553594130896942e-29
           v: [ 1.000e+00  1.000e+00 ...  1.000e+00  1.000e+00]
  optimality: 2.556280247078372e-15
        nfev: 1
        njev: 1
        allx: [Array([ 1.000e+01,  7.300e-16, ...,  0.000e+00,  0.000e+00],      dtype=float64)]
       alltr: [Array( 3.310e-01, dtype=float64)]
     history:  R_lmn: [Array([ 1.000e+01,  7.300e-16, ..., -3.751e-15,
                             -2.785e-16], dtype=float64)]
               Z_lmn: [Array([ 1.790e-15, -1.000e+00, ..., -3.313e-16,
                              1.278e-15], dtype=float64)]
               L_lmn: [Array([ 9.424e-16, -6.112e-16, ..., -5.582e-16,
                             -2.523e-16], dtype=float64)]
       

# Compute tests

In [28]:
grid = LinearGrid(L=3,M=3,N=3)

In [33]:
tranf = Transform(basis=eq.R_basis,grid=grid,method="jitable")
# jitable is direct1

In [40]:
get_transforms("B",eq,grid,jitable=True)

{'grid': LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear),
 'R': Transform at 0x293124350 (method=jitable, basis=ChebyshevZernikeBasis at 0x17f9a5890 (L=3, M=3, N=3, NFP=1, sym=False, spectral_indexing=ansi), grid=LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear)),
 'Z': Transform at 0x293195c90 (method=jitable, basis=ChebyshevZernikeBasis at 0x17f7c8590 (L=3, M=3, N=3, NFP=1, sym=False, spectral_indexing=ansi), grid=LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear)),
 'L': Transform at 0x29315acd0 (method=jitable, basis=ChebyshevZernikeBasis at 0x174a4da90 (L=3, M=3, N=3, NFP=1, sym=False, spectral_indexing=ansi), grid=LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear))}

In [42]:
eq.compute("B")["B"]

Array([[0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ],
       ...,
       [0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ]], dtype=float64)